## 04. Niches

Notebook to analyse niches<br>
Run this notebook in the 'xenium' conda env

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import anndata as ad
import scanpy as sc
import squidpy as sq
import spatialdata as sd
import spatialdata_io as sdio
import spatialdata_plot

from joblib import Parallel, delayed

from spatialdata.transformations import (
        Affine,
        Identity,
        MapAxis,
        Scale,
        Sequence,
        Translation,
        get_transformation,
        get_transformation_between_coordinate_systems,
        set_transformation,
    )

prjdir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if prjdir not in sys.path:
    sys.path.append(prjdir)

n_jobs=32
sc.settings.n_jobs=n_jobs
sc.set_figure_params(dpi=100, frameon=True, vector_friendly=True, fontsize=10)

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('grey_to_blue', ['lightgrey', 'mediumblue'])

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

/home/kirchmair/myScratch/conda/envs/crca-spatial/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
%load_ext autoreload
%autoreload 2
import src.spatial_helpers.spatial as spp
import src.spatial_helpers.spatialplot as spl
import src.spatial_helpers.sc as scp
scp.set_all_seeds()

In [ ]:
processed_datadir = '../../data/xenium/processed'
ncdir = os.path.join(processed_datadir, 'nichecompass')

## Niche analysis

In [ ]:
# load nichecompass results
adata = sc.read_h5ad(os.path.join(ncdir, 'delaunayr50', 'model', 'adata.h5ad'))

In [ ]:
# load annotations
adata_anno = sc.read_h5ad(os.path.join(processed_datadir, 'crc_ffpe_anno.h5ad'))
adata.obs = adata.obs.merge(adata_anno.obs.loc[:, ~adata_anno.obs.columns.isin(adata.obs.columns)], left_index=True, right_index=True, how='left')
adata.uns['colors'] = adata_anno.uns['colors'].copy()

In [ ]:
nc_key = 'delaunayr50_nb_leiden_0.15'
adata.obs[nc_key] = adata.obs[nc_key].astype(str)
adata.obs['CN'] = adata.obs[nc_key].astype('int') + 1
adata.obs['CN'] = 'CN' + adata.obs['CN'].astype('str')

In [ ]:
cn_colors = {
    'CN1': '#44a685',
    'CN2': '#7EC8E3',
    'CN3': '#b86c8c',
    'CN4': '#79b9f7',
    'CN5': '#6495ED',
    'CN6': '#4e6bb5',
    'CN7': '#5096e6',
    'CN8': '#3650ad',
    'CN9': '#ffd000',
    'CN10': '#4169E1',
    'CN11': '#1C39BB'
}
adata.uns['colors']['Niche'] = cn_colors

In [ ]:
# combine niches
niches_descriptive = {
    'CN1': 'Stromal niche',
    'CN2': 'Cancer - fibroblast-enriched',
    'CN3': 'Epithelial niche',
    'CN4': 'Cancer - myeloid-enriched',
    'CN5': 'Cancer',
    'CN6': 'Cancer',
    'CN7': 'Cancer',
    'CN8': 'Cancer',
    'CN9': 'Neutrophil niche',
    'CN10': 'Cancer',
    'CN11': 'Cancer'
}
adata.obs['Niche'] = adata.obs['CN'].map(niches_descriptive).astype('category')

In [ ]:
niche_colors = {
    'Epithelial niche': '#e65e5e',
    'Cancer': '#528df2',
    'Cancer - myeloid-enriched': '#6b46bd',
    'Cancer - fibroblast-enriched': '#518ebd',
    'Stromal niche': '#44a685',
    'Neutrophil niche': '#ffbf00'
}
adata.uns['colors']['Niche'] = niche_colors

## Cluster analysis

In [ ]:
nc_key = 'delaunayr50_nb_leiden_0.15'
neighbor_key='delaunayr50_connectivities'
adata.obsp['delaunayr50_connectivities'].sum(axis=1).max()

In [ ]:
celltype_neighbors_df = spp.get_neighbors(adata, obs_key='celltype', neighbors_key='delaunayr50_connectivities', n_jobs=n_jobs)

In [ ]:
# get cells that have neutrophils as neighbors
adata.obs['neutrophil_neighbors'] = celltype_neighbors_df['Neutrophil'].values.copy()

In [ ]:
# find aggregates
adata = spp.find_aggregates(adata, celltype='Neutrophil', n_neighbors=3, max_iterations=10, n_neighbors_key='neutrophil_neighbors',
                            aggr_key='neutrophil_aggregate', neighbors_key=neighbor_key)
adata.obs['neutrophil_aggregate_seed'].value_counts()
adata.obs['neutrophil_aggregate'].value_counts()

In [ ]:
adata.obs['Neutrophil'] = 'other celltype'
adata.obs.loc[adata.obs['celltype'] == 'Neutrophil', 'Neutrophil'] = 'dispersed'
adata.obs.loc[(adata.obs['celltype'] == 'Neutrophil') & (adata.obs['neutrophil_aggregate'] == True), 'Neutrophil'] = 'clustered'
adata.obs['Neutrophil'].value_counts()

In [ ]:
# save
adata.write_h5ad(os.path.join(processed_datadir, 'crc_ffpe_niches.h5ad'))

In [ ]:
sdata = sd.read_zarr(os.path.join(processed_datadir, 'crca_xenium.zarr'))
sdata['niches'] = adata
sdata = spp.match_ids(sdata, ['cell_boundaries'], table_key='anno')
sdata.tables['niches'].obs['region'] = 'cell_boundaries'
sdata.set_table_annotates_spatialelement('niches', region_key='region', region='cell_boundaries')
sdata.delete_element_from_disk('niches')
sdata.write_element('niches', overwrite=True)